# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [2]:
dfhm = pd.read_csv("scraped/hm.csv")
dfhm.head(8)

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A
5,Slub-weave Cushion Cover,$3.99 $9.99,http://www.hm.com/us/product/70965?article=70965-D
6,Braided Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/62818?article=62818-B
7,Jacquard-weave Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/69163?article=69163-B


In [3]:
dfhm['article_id'] = dfhm['url'].str.extract(r'article=(\d\d\d\d\d-[A-Z])')
# the letter in the id sets the colour of the product. That's the reason I have included it.
dfhm.head()

,name,price,url,article_id
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,74242-A


In [4]:
dfhm['price'].head()

0      $59.99 $129
1     $6.99 $17.99
2     $7.99 $17.99
3     $6.99 $12.99
4    $17.99 $24.99
Name: price, dtype: object

In [5]:
dfhm['price_original'] = dfhm['price'].str.extract(r'\$\d?\d?.\d?\d? \$(\d?\d?\d?\d?.\d?\d?\d?)').astype(float)
dfhm.head()

,name,price,url,article_id,price_original
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,13472-N,129.00
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,35079-D,17.99
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,72462-A,17.99
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,58926-C,12.99
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,74242-A,24.99


In [6]:
dfhm['price_discounted'] = dfhm['price'].str.extract(r'\$(\d?\d?.\d\d\d?\d?\d?)').astype(float)
dfhm.head()

,name,price,url,article_id,price_original,price_discounted
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,13472-N,129.00,59.99
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,35079-D,17.99,6.99
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,72462-A,17.99,7.99
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,58926-C,12.99,6.99
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,74242-A,24.99,17.99


In [7]:
dfhm['pct_discount'] = round(((dfhm['price_original'] - dfhm['price_discounted']) / dfhm['price_original']) * 100, 2)
dfhm.head()

,name,price,url,article_id,price_original,price_discounted,pct_discount
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,13472-N,129.00,59.99,53.50
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,35079-D,17.99,6.99,61.15
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,72462-A,17.99,7.99,55.59
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,58926-C,12.99,6.99,46.19
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,74242-A,24.99,17.99,28.01


In [8]:
dfhm.to_csv('hm_cleaned.csv', index=False)

## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [9]:
dfscifi = pd.read_csv("scraped/sci-fi.csv")
dfscifi.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,\nThe Handmaid's Tale\n,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"\nFrankenstein, or The Modern Prometheus\n",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"\nA Wrinkle in Time (A Wrinkle in Time Quintet, #1)\n",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,\nThe Left Hand of Darkness\n,/book/show/18423.The_Left_Hand_of_Darkness


In [10]:
dfscifi['title'] = dfscifi.title.str.replace('\n', '')

In [11]:
dfscifi.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness


In [12]:
dfscifi['avg_rating'] = dfscifi.full_rating.str.extract(r'(\d.\d\d) avg rating').astype(float)
dfscifi.head()

,full_rating,full_score,rank,title,url,avg_rating
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06


In [13]:
dfscifi['rating_count'] = dfscifi.full_rating.str.extract(r' (\d?.?\d?\d?\d?,?\d?\d?\d?) ratings')
dfscifi.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502"
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935"
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308"
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272"
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664"


In [14]:
dfscifi['total_score'] = dfscifi.full_score.str.extract(r'\nscore: (\d?\d?,?\d?\d?\d)')
dfscifi.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502","28,539"
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935","27,566"
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308","20,049"
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272","17,684"
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664","16,070"


In [15]:
dfscifi['score_votes'] = dfscifi.full_score.str.extract(r'(\d?,?\d?\d?\d) people voted').astype(int)
dfscifi.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502","28,539",292
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935","27,566",282
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308","20,049",205
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272","17,684",185
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664","16,070",165


In [16]:
dfscifi.tail()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes
95,"4.17 avg rating — 3,717 ratings","\nscore: 1,197,\n and\n16 people voted\n \n \n",96,"Invader (Foreigner, #2)",/book/show/57014.Invader,4.17,"3,717","1,197",16
96,"3.78 avg rating — 103,428 ratings","\nscore: 1,174,\n and\n13 people voted\n \n \n",97,"Across the Universe (Across the Universe, #1)",/book/show/8235178-across-the-universe,3.78,"103,428","1,174",13
97,4.16 avg rating — 97 ratings,"\nscore: 1,170,\n and\n13 people voted\n \n \n",98,"A Lost Kitten (Sea-anan Saga, #1)",/book/show/16029924-a-lost-kitten,4.16,97,"1,170",13
98,3.85 avg rating — 111 ratings,"\nscore: 1,169,\n and\n13 people voted\n \n \n",99,"Love Between the Hired Hands (The Men of Space Station One, #4)",/book/show/13482469-love-between-the-hired-hands,3.85,111,"1,169",13
99,"3.83 avg rating — 3,244 ratings","\nscore: 1,155,\n and\n16 people voted\n \n \n",100,Ammonite,/book/show/180270.Ammonite,3.83,"3,244","1,155",16


In [17]:
dfscifi['series'] = dfscifi.title.str.extract(r'\((.*),')
dfscifi.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes,series
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502","28,539",292,NaN
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935","27,566",282,The Hunger Games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308","20,049",205,NaN
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272","17,684",185,A Wrinkle in Time Quintet
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664","16,070",165,NaN


In [18]:
dfscifi['series_no'] = dfscifi.title.str.extract(r'#(\d?\d)\)')
dfscifi.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes,series,series_no
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502","28,539",292,NaN,NaN
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935","27,566",282,The Hunger Games,1
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308","20,049",205,NaN,NaN
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272","17,684",185,A Wrinkle in Time Quintet,1
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664","16,070",165,NaN,NaN


In [19]:
dfscifi.to_csv('hsci-fi.csv', index=False)

## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [20]:
# how to read in the pdfs!
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df_court = pd.DataFrame({'filename': filenames, 'content': contents})

In [21]:
df_court.head()

,filename,content
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passpor..."
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Depart..."
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse..."
3,pdfs/11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ..."
4,pdfs/11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo..."


Okay, now do the work and **make those new columns!**

In [22]:
# lawyer_app, the applicant's lawyer (pro se means that they did it themselves, that's fine)
df_court['lawyer_app'] = df_court.content.str.extract('For Applicant: (.*)', re.IGNORECASE)
df_court['lawyer_app']

0                      Pro se 
1                          NaN
2          Mark S. Zaid, Esq. 
3                      Pro se 
4                      Pro se 
5                      Pro se 
6                      Pro se 
7                      Pro se 
8     Ryan C. Nerney, Esquire 
9                      Pro Se 
10                     Pro se 
11                         NaN
12                      Pro se
13         Mark S. Zaid, Esq. 
14     Stephen Glassman, Esq. 
Name: lawyer_app, dtype: object

In [23]:
# lawyer_gov, the government's lawyer

df_court['lawyer_gov'] = df_court.content.str.extract('For Government: (.*)', re.IGNORECASE)
df_court['lawyer_gov']

0          David F. Hayes, Esq., Department Counsel 
1                                                NaN
2     Robert J. Kilmartin, Esq., Department Counsel 
3         Richard Stevens, Esq., Department Counsel 
4      Julie R. Mendez, Esquire, Department Counsel 
5       David F. Hayes, Esquire, Department Counsel 
6       Stephanie C. Hess, Esq., Department Counsel 
7            Tara Karoian, Esq., Department Counsel 
8      Richard Stevens, Esquire, Department Counsel 
9        Robert Kilmartin, Esq., Department Counsel 
10      Eric Borgstrom, Esquire, Department Counsel 
11                                               NaN
12        Tovah Minster, Esquire, Department Counsel
13        Julie R. Mendez, Esq., Department Counsel 
14          Erin Thompson, Esq., Department Counsel 
Name: lawyer_gov, dtype: object

In [24]:
# judge, the name of the judge

df_court['judge'] = df_court.content.str.extract('([A-Z]+.*), Administrative', re.IGNORECASE)
df_court['judge'].head()

0           MENDEZ, Francisco
1                         NaN
2         LOUGHRAN, Edward W.
3             DUFFY, James F.
4    MARSHALL, Jr., Arthur E.
Name: judge, dtype: object

In [25]:
# access, whether the clearance is granted or denied (although you might miss a few)
df_court['access'] = df_court['content'].str.contains(r'is granted')
df_court['access'].head()

0    False
1    False
2     True
3     True
4    False
Name: access, dtype: bool

In [26]:
df_court['has_debt'] = df_court['content'].str.contains(r'debt')
df_court['has_debt'].head()

0    False
1     True
2     True
3     True
4    False
Name: has_debt, dtype: bool

In [27]:
df_court['has_bankruptcy'] = df_court['content'].str.contains(r'bankruptcy')
df_court['has_bankruptcy'].head()

0    False
1     True
2     True
3    False
4    False
Name: has_bankruptcy, dtype: bool

In [28]:
df_court

,filename,content,lawyer_app,lawyer_gov,judge,access,has_debt,has_bankruptcy
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passpor...",Pro se,"David F. Hayes, Esq., Department Counsel","MENDEZ, Francisco",False,False,False
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Depart...",NaN,NaN,NaN,False,True,True
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse...","Mark S. Zaid, Esq.","Robert J. Kilmartin, Esq., Department Counsel","LOUGHRAN, Edward W.",True,True,True
3,pdfs/11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ...",Pro se,"Richard Stevens, Esq., Department Counsel","DUFFY, James F.",True,True,False
4,pdfs/11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo...",Pro se,"Julie R. Mendez, Esquire, Department Counsel","MARSHALL, Jr., Arthur E.",False,False,False
5,pdfs/12-01969.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \n \nIn the matter of: \n\n----------------------------- \n \n\nApplicant for Security Clearance \n\n) \n) \n) \n) \n) \n) \n \n \n\n ISCR Case No. 12-01969 \n\nAppearances \n\nFor Government: David F. Hayes, Esquire, Department Counsel \n\nFor Applicant: Pro se \n\n \n \n\n______________ \n\n \n\nDecisi...",Pro se,"David F. Hayes, Esquire, Department Counsel","HOWE, Philip S.",False,True,False
6,pdfs/11-02438.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-02438 \n\nFor Government: Stephanie C. Hess, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\...",Pro se,"Stephanie C. Hess, Esq., Department Counsel","COACHER, Robert E.",False,False,False
7,pdfs/11-14135.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-14135 \n\nAppearances \n\nFor Government: Tara Karoian, Esq., Department

In [29]:
df_court.to_csv('court_cleaned.csv', index=False)

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [30]:
contents = open("data/Austen_Pride.txt").read()

## Look at the first 500 or so characters of it 

In [31]:
contents[:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [32]:
import re

matches = re.findall(r"\bhe\b|\bshe\b", contents, re.IGNORECASE)

# Count the number of 'he' and 'she'
print(len(matches))

print(matches)

3047
['he', 'he', 'she', 'she', 'he', 'he', 'he', 'he', 'he', 'he', 'she', 'he', 'he', 'she', 'he', 'She', 'she', 'she', 'He', 'he', 'she', 'he', 'She', 'She', 'she', 'she', 'she', 'he', 'he', 'he', 'she', 'she', 'he', 'he', 'she', 'he', 'he', 'He', 'he', 'He', 'he', 'he', 'he', 'She', 'he', 'she', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'He', 'he', 'he', 'She', 'he', 'he', 'She', 'She', 'she', 'she', 'he', 'he', 'He', 'he', 'he', 'she', 'she', 'he', 'she', 'he', 'he', 'he', 'he', 'she', 'he', 'she', 'he', 'he', 'he', 'he', 'He', 'she', 'She', 'she', 'he', 'He', 'he', 'she', 'He', 'she', 'He', 'he', 'He', 'he', 'she', 'he', 'he', 'he', 'he', 'He', 'he', 'He', 'he', 'he', 'he', 'he', 'he', 'she', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'she', 'he', 'he', 'she', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'he', 'she', 'she', 'she', 'he', 'she', 'she', 'She', 'she', 'she', 'he', 'she', 'she', 'he', 'he', 'he', 'he', 'he',

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [33]:
matches = re.findall(r"\bhe\b \w*|\bshe\b \w*", contents, re.IGNORECASE)
print(matches)

['he is', 'he had', 'she told', 'he came', 'he agreed', 'he is', 'he married', 'he may', 'he comes', 'she ought', 'he comes', 'he chooses', 'she is', 'She was', 'she was', 'she fancied', 'He had', 'he should', 'she had', 'he suddenly', 'She has', 'She is', 'she times', 'she will', 'she will', 'he continued', 'he wished', 'she began', 'she had', 'he spoke', 'he left', 'he would', 'he eluded', 'He was', 'he meant', 'He had', 'he had', 'he saw', 'he wore', 'She could', 'he could', 'she began', 'he might', 'he ought', 'he brought', 'he had', 'he was', 'he was', 'he was', 'he was', 'He was', 'he would', 'She is', 'he looked', 'he withdrew', 'She is', 'She told', 'she had', 'she had', 'he was', 'he had', 'He had', 'he ', 'he had', 'she entered', 'she looked', 'he actually', 'she was', 'he asked', 'he asked', 'he did', 'he seemed', 'she was', 'he inquired', 'she was', 'he danced', 'he had', 'he would', 'he had', 'He is', 'she was', 'She was', 'she added', 'he is', 'He walked', 'he walked', 's

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [34]:
matches = re.findall(r"(\bs?he\b) (\w*)", contents, re.IGNORECASE)
print(matches)

[('he', 'is'), ('he', 'had'), ('she', 'told'), ('he', 'came'), ('he', 'agreed'), ('he', 'is'), ('he', 'married'), ('he', 'may'), ('he', 'comes'), ('she', 'ought'), ('he', 'comes'), ('he', 'chooses'), ('she', 'is'), ('She', 'was'), ('she', 'was'), ('she', 'fancied'), ('He', 'had'), ('he', 'should'), ('she', 'had'), ('he', 'suddenly'), ('She', 'has'), ('She', 'is'), ('she', 'times'), ('she', 'will'), ('she', 'will'), ('he', 'continued'), ('he', 'wished'), ('she', 'began'), ('she', 'had'), ('he', 'spoke'), ('he', 'left'), ('he', 'would'), ('he', 'eluded'), ('He', 'was'), ('he', 'meant'), ('He', 'had'), ('he', 'had'), ('he', 'saw'), ('he', 'wore'), ('She', 'could'), ('he', 'could'), ('she', 'began'), ('he', 'might'), ('he', 'ought'), ('he', 'brought'), ('he', 'had'), ('he', 'was'), ('he', 'was'), ('he', 'was'), ('he', 'was'), ('He', 'was'), ('he', 'would'), ('She', 'is'), ('he', 'looked'), ('he', 'withdrew'), ('She', 'is'), ('She', 'told'), ('she', 'had'), ('she', 'had'), ('he', 'was'), ('

## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [35]:
new_df = pd.DataFrame(matches, columns=['pronoun', 'verb'])
new_df.head()

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed


## How many times is each pronoun used?

In [36]:
new_df['pronoun'].value_counts()

she    1324
he     1056
She     325
He      234
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [37]:
new_df['pronoun'] = new_df['pronoun'].str.lower()
new_df['pronoun'].value_counts()

she    1649
he     1290
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [38]:
new_df['verb'].value_counts().head(20)

was          372
had          371
could        172
is           139
would         94
has           70
did           67
will          50
might         46
should        41
felt          38
must          37
said          33
saw           32
added         31
thought       31
then          26
replied       22
continued     21
looked        21
Name: verb, dtype: int64

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [39]:
# HE
new_df[new_df['pronoun'] == 'he']['verb'].value_counts().head(20)

had        166
was        160
is          74
has         49
could       40
would       35
did         29
should      26
must        24
will        24
might       21
replied     14
thought     13
may         13
came        13
said        13
does        12
never       12
then        11
meant       11
Name: verb, dtype: int64

In [40]:
# SHE
new_df[new_df['pronoun'] == 'she']['verb'].value_counts().head(20)

was        212
had        205
could      132
is          65
would       59
did         38
felt        33
saw         29
will        26
might       25
added       23
has         21
said        20
thought     18
then        15
should      15
found       13
must        13
soon        12
spoke       11
Name: verb, dtype: int64

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [41]:
new_df[new_df['verb'] == 'cried']['pronoun'].value_counts(normalize=True)

she    0.916667
he     0.083333
Name: pronoun, dtype: float64

## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [42]:
tolkien = open('data/Lord of the Rings - 01 - The Fellowship of the Ring - J. R. R. Tolkien - 1955.txt').read()


In [43]:
tolkien_heshe = re.findall(r'\bs?he\b', tolkien, re.IGNORECASE)
tolkien_heshe = pd.DataFrame(tolkien_heshe, columns=['pronoun'])
tolkien_heshe['pronoun'] = tolkien_heshe['pronoun'].str.lower()

In [44]:
# Percentage of 'he' and 'she' in J.R.R. Tolkein's Fellowship of the Ring
tolkien_heshe['pronoun'].value_counts(normalize=True)

he     0.950636
she    0.049364
Name: pronoun, dtype: float64

In [45]:
# Pride and Prejudice
new_df['pronoun'].value_counts(normalize=True)


she    0.561075
he     0.438925
Name: pronoun, dtype: float64

In [46]:
print("Fellowship of the Ring: 'He' is", round(tolkien_heshe['pronoun'].value_counts()[0] / tolkien_heshe['pronoun'].value_counts()[1], 2), "times more common")

Fellowship of the Ring: 'He' is 19.26 times more common


In [47]:
print("Pride and Prejudice: 'She' is", round(new_df['pronoun'].value_counts()[0] / new_df['pronoun'].value_counts()[1], 2), "times more common")

Pride and Prejudice: 'She' is 1.28 times more common
